In [20]:
from datetime import datetime, timedelta
import pandas as pd
import urllib.request
import json

In [21]:
api_key = "98716afd22b34d428b2145552231708" 

In [22]:
def retrieve_hist_data(api_key, location_list, start_date, end_date, frequency, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None):
    result_list = []
    for location in location_list:
        print('\n\nRetrieving weather data for ' + location + '\n\n')
        df_this_city = retrieve_this_location(api_key, location, start_date, end_date, frequency, response_cache_path)

        if (location_label == True):
            # add city name as prefix to the colnames
            df_this_city = df_this_city.add_prefix(location + '_')
            df_this_city.columns.values[0] = 'date_time'

        if (export_csv == True):
            df_this_city.to_csv('./' + location + '.csv', header=True, index=False)
            print('\n\nexport ' + location + ' completed!\n\n')

        if (store_df == True):
            # save result as object in the work space
            result_list.append(df_this_city)

    return (result_list)

In [26]:
def retrieve_this_location(api_key, location, start_date, end_date, frequency, response_cache_path):
    start_time = datetime.now()

    # create list of first day of month for range between start and end dates non-inclusive (open)
    list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
    # convert to Series and add start_date at beginning
    list_mon_begin = pd.concat([pd.Series(pd.to_datetime(start_date)), pd.Series(list_mon_begin)], ignore_index=True)

    # create list of month end dates for range between start and end dates non-inclusive (open)
    list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')
    # convert to Series and add end_date at end
    list_mon_end = pd.concat([pd.Series(list_mon_end), pd.Series(pd.to_datetime(end_date))], ignore_index=True)

    # count number of months to be retrieved
    total_months = len(list_mon_begin)

    # initialize df_hist to store return data
    df_hist = pd.DataFrame()
    for m in range(total_months):
        start_d = str(list_mon_begin[m])[:10]
        end_d = str(list_mon_end[m])[:10]
        file_path = f'{response_cache_path}/{location}_{start_d}_{end_d}'
        
        try:
            if response_cache_path and os.path.exists(file_path):
                print('Reading cached data for ' + location + ': from ' + start_d + ' to ' + end_d)
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
            else:
                print('Currently retrieving data for ' + location + ': from ' + start_d + ' to ' + end_d)
                url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + api_key + '&q=' + location + '&format=json&date=' + start_d + '&enddate=' + end_d + '&tp=' + str(frequency)
                json_page = urllib.request.urlopen(url_page, timeout=10)
                json_data = json.loads(json_page.read().decode())
                
                if response_cache_path:
                    with open(file_path, 'w') as f:
                        json.dump(json_data, f)
            
            data = json_data['data']['weather']
            # call function to extract json object
            df_this_month = extract_monthly_data(data)
            df_this_month['location'] = location
            df_hist = pd.concat([df_hist, df_this_month])
        except Exception as e:
            print('Error encountered:', e)

        time_elapsed = datetime.now() - start_time
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    
    return df_hist

In [27]:
def extract_monthly_data(data):
    num_days = len(data)
    # initialize df_month to store return data
    df_month = pd.DataFrame()
    for i in range(num_days):
        # extract this day
        d = data[i]
        # astronomy data is the same for the whole day
        astr_df = pd.DataFrame(d['astronomy'])
        # hourly data; temperature for each hour of the day
        hourly_df = pd.DataFrame(d['hourly'])
        # this wanted_key will be duplicated and use 'ffill' to fill up the NAs
        wanted_keys = ['date', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex']  # The keys you want
        subset_d = dict((k, d[k]) for k in wanted_keys if k in d)
        this_df = pd.DataFrame(subset_d, index=[0])
        df = pd.concat([this_df.reset_index(drop=True), astr_df], axis=1)
        # concat selected astonomy columns with hourly data
        df = pd.concat([df, hourly_df], axis=1)
        df = df.fillna(method='ffill')
        # make date_time columm to proper format
        # fill leading zero for hours to 4 digits (0000-2400 hr)
        df['time'] = df['time'].apply(lambda x: x.zfill(4))
        # keep only first 2 digit (00-24 hr) 
        df['time'] = df['time'].str[:2]
        # convert to pandas datetime
        df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        # keep only interested columns
        col_to_keep = ['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex',
                       'moon_illumination', 'moonrise', 'moonset', 'sunrise', 'sunset',
                       'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph',
                       'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
                       'winddirDegree', 'windspeedKmph']
        df = df[col_to_keep]
        df = df.loc[:,~df.columns.duplicated()]
        df_month = pd.concat([df_month, df])
    return (df_month)

In [31]:
location_list = ["bellville-south-weather", "johannesburg-weather", "lawley-south-weather", "ma-africa-weather","pretoria-weather"]
start_date = "2009-01-01"
end_date = "2021-12-31"

# Initialize lists to store data

retrieve_hist_data(api_key, location_list, start_date, end_date, frequency=24)



Retrieving weather data for bellville-south-weather


Currently retrieving data for bellville-south-weather: from 2009-01-01 to 2009-01-31


/tmp/ipykernel_99479/909374209.py:5: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
/tmp/ipykernel_99479/909374209.py:10: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:02.962872
Currently retrieving data for bellville-south-weather: from 2009-02-01 to 2009-02-28
Time elapsed (hh:mm:ss.ms) 0:00:05.352425
Currently retrieving data for bellville-south-weather: from 2009-03-01 to 2009-03-31
Time elapsed (hh:mm:ss.ms) 0:00:07.757416
Currently retrieving data for bellville-south-weather: from 2009-04-01 to 2009-04-30
Time elapsed (hh:mm:ss.ms) 0:00:09.383525
Currently retrieving data for bellville-south-weather: from 2009-05-01 to 2009-05-31
Time elapsed (hh:mm:ss.ms) 0:00:11.185830
Currently retrieving data for bellville-south-weather: from 2009-06-01 to 2009-06-30
Time elapsed (hh:mm:ss.ms) 0:00:13.218567
Currently retrieving data for bellville-south-weather: from 2009-07-01 to 2009-07-31
Time elapsed (hh:mm:ss.ms) 0:00:14.789388
Currently retrieving data for bellville-south-weather: from 2009-08-01 to 2009-08-31
Time elapsed (hh:mm:ss.ms) 0:00:16.968765
Currently retrieving data for bellville-south-weather: from 2009-09-0

/tmp/ipykernel_99479/909374209.py:5: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
/tmp/ipykernel_99479/909374209.py:10: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:02.351227
Currently retrieving data for johannesburg-weather: from 2009-02-01 to 2009-02-28
Time elapsed (hh:mm:ss.ms) 0:00:04.448460
Currently retrieving data for johannesburg-weather: from 2009-03-01 to 2009-03-31
Time elapsed (hh:mm:ss.ms) 0:00:06.860313
Currently retrieving data for johannesburg-weather: from 2009-04-01 to 2009-04-30
Time elapsed (hh:mm:ss.ms) 0:00:08.741488
Currently retrieving data for johannesburg-weather: from 2009-05-01 to 2009-05-31
Time elapsed (hh:mm:ss.ms) 0:00:10.301058
Currently retrieving data for johannesburg-weather: from 2009-06-01 to 2009-06-30
Time elapsed (hh:mm:ss.ms) 0:00:11.824836
Currently retrieving data for johannesburg-weather: from 2009-07-01 to 2009-07-31
Time elapsed (hh:mm:ss.ms) 0:00:13.638914
Currently retrieving data for johannesburg-weather: from 2009-08-01 to 2009-08-31
Time elapsed (hh:mm:ss.ms) 0:00:15.637584
Currently retrieving data for johannesburg-weather: from 2009-09-01 to 2009-09-30
Time ela

...

/tmp/ipykernel_99479/909374209.py:5: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
/tmp/ipykernel_99479/909374209.py:10: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:02.402078
Currently retrieving data for ma-africa-weather: from 2009-02-01 to 2009-02-28
Time elapsed (hh:mm:ss.ms) 0:00:04.335703
Currently retrieving data for ma-africa-weather: from 2009-03-01 to 2009-03-31
Time elapsed (hh:mm:ss.ms) 0:00:06.844049
Currently retrieving data for ma-africa-weather: from 2009-04-01 to 2009-04-30
Time elapsed (hh:mm:ss.ms) 0:00:08.710330
Currently retrieving data for ma-africa-weather: from 2009-05-01 to 2009-05-31
Time elapsed (hh:mm:ss.ms) 0:00:10.337744
Currently retrieving data for ma-africa-weather: from 2009-06-01 to 2009-06-30
Time elapsed (hh:mm:ss.ms) 0:00:12.457135
Currently retrieving data for ma-africa-weather: from 2009-07-01 to 2009-07-31
Time elapsed (hh:mm:ss.ms) 0:00:14.282204
Currently retrieving data for ma-africa-weather: from 2009-08-01 to 2009-08-31
Time elapsed (hh:mm:ss.ms) 0:00:15.838489
Currently retrieving data for ma-africa-weather: from 2009-09-01 to 2009-09-30
Time elapsed (hh:mm:ss.ms) 0:00:

/tmp/ipykernel_99479/909374209.py:5: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
/tmp/ipykernel_99479/909374209.py:10: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:02.083967
Currently retrieving data for pretoria-weather: from 2009-02-01 to 2009-02-28
Time elapsed (hh:mm:ss.ms) 0:00:04.721530
Currently retrieving data for pretoria-weather: from 2009-03-01 to 2009-03-31
Time elapsed (hh:mm:ss.ms) 0:00:07.086305
Currently retrieving data for pretoria-weather: from 2009-04-01 to 2009-04-30
Time elapsed (hh:mm:ss.ms) 0:00:08.805189
Currently retrieving data for pretoria-weather: from 2009-05-01 to 2009-05-31
Time elapsed (hh:mm:ss.ms) 0:00:12.675403
Currently retrieving data for pretoria-weather: from 2009-06-01 to 2009-06-30
Time elapsed (hh:mm:ss.ms) 0:00:14.429557
Currently retrieving data for pretoria-weather: from 2009-07-01 to 2009-07-31
Time elapsed (hh:mm:ss.ms) 0:00:15.965536
Currently retrieving data for pretoria-weather: from 2009-08-01 to 2009-08-31
Time elapsed (hh:mm:ss.ms) 0:00:17.716477
Currently retrieving data for pretoria-weather: from 2009-09-01 to 2009-09-30
Time elapsed (hh:mm:ss.ms) 0:00:19.21709

[]